# TP2: Machine learning

In [58]:
import pandas as pd
%matplotlib inline
from datetime import datetime
import scipy.spatial
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [59]:
data_training = pd.read_csv('data/properati-merge2017-2016-2015-SOLOSIX.csv')

In [60]:
data_training = data_training[data_training['state_name'].str.contains('G.B.A') | data_training['state_name'].str.contains('Capital Federal')]
##El tp dice solo gba y caba

In [61]:
def recuperar_cant_amb(x):
    "Recupera el dato amb en x (puede ser la descripcion o el titulo)"
    try:
        x = x.lower()
        if x.rfind("ambientes") != -1 or x.rfind("amb") != -1:
            pos = x.rfind("amb")
            try:
                return (float(x[pos -2])) # intento convertir lo que hay antes de la palabra
                                              #si funciona es el numero de rooms
            except ValueError:
                if "dos amb" in x:
                    return 2.0
                elif "tres amb" in x:
                    return 3.0
                elif "cuatro amb" in x:
                    return 4.0
                elif "cinco amb" in x:
                    return 5.0
                elif "seis amb" in x:
                    return 6.0
                return 0
    except AttributeError:
        return None

In [62]:
def buscar_servicios_en_descripcion(descripcion):
    """Devuelve un diccionario con 1 o 0 segun cuente con el servicio o no.
    Los servicios que consideraremos son cochera,pileta, vigilancia y gimnasio"""
    dic = {"cochera":0,"pileta":0,"vigilancia":0,"gimnasio":0,"patio":0,"terraza":0,"parrilla":0,"lavadero":0}
    descripcion = str(descripcion)
    descripcion = str.lower(descripcion)
    if "cochera" in descripcion or "garage" in descripcion:
        dic["cochera"] = 1
    elif "pileta" in descripcion or "piscina" in descripcion:
        dic["pileta"] = 1
    elif "vigilancia" in descripcion:
        dic["vigilancia"] = 1
    elif "gimnasio" in descripcion:
        dic["gimnasio"] = 1
    elif "patio" in descripcion:
        dic["patio"] = 1
    elif "terraza" in descripcion:
        dic["terraza"] = 1
    elif "parrilla" in descripcion:
        dic["parrilla"] = 1
    elif "lavadero" in descripcion:
        dic["lavadero"] = 1
    return dic

In [63]:
def contar_ambientes(x):
    "Define la cantidad de ambientes en la descripcion"
    contador = 0
    try:
        x = x.lower()
        if x.rfind("dormitorios") != -1: 
            pos = x.rfind("dormitorios")
        elif x.rfind("habitaciones") != -1:
            pos = x.rfind("habitaciones")
            try:
                contador += (float(x[pos -2])) # intento convertir lo que hay antes de la palabra
                                              #si funciona es el numero de rooms
            except ValueError:
                if ("dos dormitorios" in x) or ("dos habitaciones" in x):
                    contador += 2.0
                elif ("tres dormitorios" in x) or ("tres habitaciones" in x):
                    contador += 3.0
                elif ("cuatro dormitorios" in x) or ("cuatro habitaciones" in x):
                    contador += 4.0
                elif ("cinco dormitorios" in x) or ("cinco habitaciones" in x):
                    contador += 5.0
                elif ("seis dormitorios" in x) or ("seis habitaciones" in x):
                    contador += 6.0
        if ("living" or "comedor") in x:
            contador += 1.0
    except AttributeError:
        return None
    return contador

In [64]:
def place_name_from_parent(x): ## Segun vi el 3 item del place es generalmente el lugar
    list = x.split('|')
    if(len(list) < 5):
        return list[2]
    else:
        return list[3]

In [65]:
#Hay registros con precio en usd 0, pero en su descripcion se encuentra el precio asi que lo rescatamos
def rescatar_precio_descrip(descripcion):
    try:
        descripcion = str.lower(descripcion)
    except TypeError:
        #No tiene descripcion
        return 0
    try:
        if descripcion.rfind("usd") != -1 and descripcion.rfind("null") != -1:
            pos_usd = descripcion.rfind("usd")
            pos_null = descripcion.rfind("null")
            precio = (descripcion[pos_usd + 4 : pos_null -1]).lstrip().rstrip()
            if precio.isnumeric():
                return float(precio)
            elif descripcion.rfind('codigo: 2007-60') != -1:
                #Es un registro que no puedo recuperar genericamente
                return 35000
            else:
                #No es del formato USD precio null

                return 0
    except AttributeError:
        return 0

In [66]:
#Inicializo columnas
data_training["servicios"] = np.nan
data_training["cochera"] = int(0)
data_training["pileta"] = int(0)
data_training["vigilancia"] = int(0)
data_training["gimnasio"] = int(0)
data_training["patio"] = int(0)
data_training["terraza"] = int(0)
data_training["parrilla"] = int(0)
data_training["lavadero"] = int(0)

In [67]:
#Busco en las descripciones
data_training.loc[data_training["description"].notnull(),"servicios"] = data_training[data_training["description"].notnull()].description.apply(buscar_servicios_en_descripcion)

In [68]:
#Modifico los valores encontrados en la tabla
data_training.loc[:,"cochera"] = data_training[data_training["servicios"].notnull()].apply(lambda x: x.servicios["cochera"],axis = 1)
data_training.loc[:,"pileta"] = data_training[data_training["servicios"].notnull()].apply(lambda x: x.servicios["pileta"],axis=1)
data_training.loc[:,"vigilancia"] = data_training[data_training["servicios"].notnull()].apply(lambda x: x.servicios["vigilancia"],axis=1)
data_training.loc[:,"gimnasio"] = data_training[data_training["servicios"].notnull()].apply(lambda x: x.servicios["gimnasio"],axis=1)
data_training.loc[:,"patio"] = data_training[data_training["servicios"].notnull()].apply(lambda x: x.servicios["patio"],axis = 1)
data_training.loc[:,"terraza"] = data_training[data_training["servicios"].notnull()].apply(lambda x: x.servicios["terraza"],axis=1)
data_training.loc[:,"parrilla"] = data_training[data_training["servicios"].notnull()].apply(lambda x: x.servicios["parrilla"],axis=1)
data_training.loc[:,"lavadero"] = data_training[data_training["servicios"].notnull()].apply(lambda x: x.servicios["lavadero"],axis=1)

In [69]:
#Hay 12 campos que no se porque no se rellenan con 0
data_training.cochera.fillna(0,inplace=True)
data_training.pileta.fillna(0,inplace=True)
data_training.vigilancia.fillna(0,inplace=True)
data_training.gimnasio.fillna(0,inplace=True)
data_training.patio.fillna(0,inplace=True)
data_training.terraza.fillna(0,inplace=True)
data_training.parrilla.fillna(0,inplace=True)
data_training.lavadero.fillna(0,inplace=True)

In [70]:
#Elimino la columna servicios que ya no me sirve
data_training.drop("servicios",1,inplace=True)
data_training.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264021 entries, 0 to 264020
Data columns (total 31 columns):
id                            264021 non-null object
created_on                    264021 non-null object
property_type                 264021 non-null object
place_name                    263974 non-null object
place_with_parent_names       264021 non-null object
state_name                    264021 non-null object
geonames_id                   211642 non-null float64
lat-lon                       188028 non-null object
lat                           188028 non-null float64
lon                           188028 non-null float64
price                         246208 non-null float64
currency                      238346 non-null object
price_aprox_local_currency    246208 non-null float64
price_aprox_usd               246208 non-null float64
surface_total_in_m2           189637 non-null float64
surface_covered_in_m2         231150 non-null float64
price_usd_per_m2              160

In [71]:
data_training.loc[data_training['price_aprox_usd'] == 0,'price_aprox_usd'] = data_training.loc[data_training['price_aprox_usd'] == 0].description.apply(rescatar_precio_descrip)

In [72]:
data_training.dropna(axis=0,subset=['price_aprox_usd'],inplace=True) ## drop de los datos sin precio
data_training = data_training.loc[data_training['price_aprox_usd'] > 0,:]

#Recuperamos el dato rooms en el titulo y la descripcion
data_training.loc[data_training.rooms.isnull(),'rooms'] = data_training.loc[data_training.rooms.isnull()].description.apply(recuperar_cant_amb)
data_training.loc[data_training.rooms.isnull(),'rooms'] = data_training.loc[data_training.rooms.isnull()].title.apply(recuperar_cant_amb)
#Recuperamos el dato rooms en la descripcion contando ambientes (living y dormitorios)
data_training.loc[data_training.rooms.isnull(),'rooms'] = data_training.loc[data_training.rooms.isnull()].description.apply(contar_ambientes)
data_training.loc[data_training.rooms == 0,'rooms'] = data_training.loc[data_training.rooms == 0].description.apply(contar_ambientes)


data_training.loc[:,'place_name'] = data_training.place_with_parent_names.apply(place_name_from_parent)
## esto quita muchos problemas de place name como nombres de country o palermo soho

In [73]:
data_training = data_training.dropna(axis = 0, subset = ['price_aprox_usd','surface_total_in_m2'],how ='any')
data_training['year'] = 0
data_training['month'] = 0
data_training.loc[:,'created_on'] = data_training.created_on.apply(pd.to_datetime)

In [74]:
data_training.loc[:,'year'] = data_training.created_on.apply(lambda x: x.year)
data_training.loc[:,'month'] = data_training.created_on.apply(lambda x: x.month)
data_training.drop('created_on',1,inplace=True)

In [75]:
#Hay 2 registro con rooms nan irrecuperables
data_training.drop(132299,inplace=True)
data_training.drop(95386,inplace=True)

In [76]:
#NO considerare lat lon porque reduce mucho los registros
data_training.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171040 entries, 0 to 264017
Data columns (total 32 columns):
id                            171040 non-null object
property_type                 171040 non-null object
place_name                    171040 non-null object
place_with_parent_names       171040 non-null object
state_name                    171040 non-null object
geonames_id                   136645 non-null float64
lat-lon                       121146 non-null object
lat                           121146 non-null float64
lon                           121146 non-null float64
price                         171040 non-null float64
currency                      171040 non-null object
price_aprox_local_currency    171040 non-null float64
price_aprox_usd               171040 non-null float64
surface_total_in_m2           171040 non-null float64
surface_covered_in_m2         162968 non-null float64
price_usd_per_m2              160036 non-null float64
price_per_m2                  15

In [77]:
#Recuperamos registros a los que no se les calculo el precio por m2 en dolares
data_training.loc[data_training['price_usd_per_m2'].isnull(),'price_usd_per_m2'] = data_training.surface_total_in_m2 / data_training.price_aprox_usd
data_training.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171040 entries, 0 to 264017
Data columns (total 32 columns):
id                            171040 non-null object
property_type                 171040 non-null object
place_name                    171040 non-null object
place_with_parent_names       171040 non-null object
state_name                    171040 non-null object
geonames_id                   136645 non-null float64
lat-lon                       121146 non-null object
lat                           121146 non-null float64
lon                           121146 non-null float64
price                         171040 non-null float64
currency                      171040 non-null object
price_aprox_local_currency    171040 non-null float64
price_aprox_usd               171040 non-null float64
surface_total_in_m2           171040 non-null float64
surface_covered_in_m2         162968 non-null float64
price_usd_per_m2              171040 non-null float64
price_per_m2                  15

In [78]:
#Vamos a codificar cada columna que sea de strings
lb_make1 = LabelEncoder()
data_training["property_type_code"] = lb_make1.fit_transform(data_training["property_type"])

lb_make2 = LabelEncoder()
data_training["place_name_code"] = lb_make2.fit_transform(data_training["place_name"])

lb_make3 = LabelEncoder()
data_training["place_with_parent_names_code"] = lb_make3.fit_transform(data_training["place_with_parent_names"])

lb_make4 = LabelEncoder()
data_training["state_name_code"] = lb_make4.fit_transform(data_training["state_name"])

In [81]:
data_training2 = data_training[['lat', 'lon', 'price_aprox_usd', 'surface_total_in_m2',
       'price_usd_per_m2', 'rooms', 'year', 'month', 'property_type_code',
       'place_name_code', 'place_with_parent_names_code', 'state_name_code']]

data_training2 = data_training2.dropna(axis = 0, subset = ['lat','lon'],how ='any')

In [82]:
data_training3 = data_training[['price_aprox_usd','lat','lon','surface_total_in_m2',
       'rooms', 'year', 'month', 'property_type_code','place_name_code', 
        'place_with_parent_names_code', 'state_name_code','cochera', 'pileta', 'vigilancia',
        'gimnasio', 'patio', 'terraza','parrilla', 'lavadero']]
data_training3 = data_training3.dropna(axis = 0, subset = ['lat','lon'],how ='any')

In [83]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import HuberRegressor 
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LassoLars
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import BayesianRidge

dataframe = data_training3.drop(['price_aprox_usd'],axis=1)

columnas = list(dataframe.columns)
columnas_precio = columnas + ['price_aprox_usd']

set_entrenamiento = data_training3.loc[(data_training3.year >= 2014) &((data_training3.year < 2017) | (data_training3.month < 6)),columnas_precio]
set_pruebas = data_training3.loc[(data_training3.year == 2017) & (data_training3.month == 6),columnas_precio]

array = set_entrenamiento.values
X = array[:,0:18]
Y = array[:,18]

# prepare configuration for cross validation test harness
seed = 7
# prepare models
models = []
models.append(('Linear Regression', LinearRegression()))
models.append(('Ridge', Ridge()))
models.append(('RidgeCV', RidgeCV()))
models.append(('Lasso', Lasso()))
models.append(('Elastic Net', ElasticNet()))
models.append(('LassoLars', LassoLars()))
models.append(('OrthogonalMatchingPursuit', OrthogonalMatchingPursuit()))
models.append(('BayesianRidge', BayesianRidge()))
models.append(('HuberRegressor  ', HuberRegressor()))

# evaluate each model in turn
for name, model in models:
    model.fit(X,Y)
    score = model.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
    print("Precision = {:.2f} % , name = {}".format(score,name))


Precision = 14.94 % , name = Linear Regression
Precision = 14.94 % , name = Ridge
Precision = 14.95 % , name = RidgeCV
Precision = 14.94 % , name = Lasso
Precision = 11.42 % , name = Elastic Net
Precision = 14.95 % , name = LassoLars
Precision = 3.07 % , name = OrthogonalMatchingPursuit
Precision = 14.95 % , name = BayesianRidge
Precision = 6.44 % , name = HuberRegressor  


In [84]:
trainingSet = data_training3
testingSet = pd.read_csv("data/properati_dataset_testing_noprice.csv")

In [85]:
data_training3.columns

Index([u'price_aprox_usd', u'lat', u'lon', u'surface_total_in_m2', u'rooms',
       u'year', u'month', u'property_type_code', u'place_name_code',
       u'place_with_parent_names_code', u'state_name_code', u'cochera',
       u'pileta', u'vigilancia', u'gimnasio', u'patio', u'terraza',
       u'parrilla', u'lavadero'],
      dtype='object')

# Formateo el testing set

In [86]:
testingSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 17 columns):
id                         14166 non-null int64
created_on                 14166 non-null object
property_type              14166 non-null object
operation                  14166 non-null object
place_name                 14166 non-null object
place_with_parent_names    14166 non-null object
country_name               14166 non-null object
state_name                 14166 non-null object
lat-lon                    10487 non-null object
lat                        10487 non-null float64
lon                        10487 non-null float64
surface_total_in_m2        11853 non-null float64
surface_covered_in_m2      13005 non-null float64
floor                      1368 non-null float64
rooms                      7500 non-null float64
expenses                   2543 non-null object
description                14166 non-null object
dtypes: float64(6), int64(1), object(10)
memory usage: 1

In [87]:
#Inicializo columnas
testingSet["servicios"] = np.nan
testingSet["cochera"] = int(0)
testingSet["pileta"] = int(0)
testingSet["vigilancia"] = int(0)
testingSet["gimnasio"] = int(0)
testingSet["patio"] = int(0)
testingSet["terraza"] = int(0)
testingSet["parrilla"] = int(0)
testingSet["lavadero"] = int(0)

In [88]:
#Busco en las descripciones
testingSet.loc[testingSet["description"].notnull(),"servicios"] = testingSet[testingSet["description"].notnull()].description.apply(buscar_servicios_en_descripcion)

In [89]:
#Modifico los valores encontrados en la tabla
testingSet.loc[:,"cochera"] = testingSet[testingSet["servicios"].notnull()].apply(lambda x: x.servicios["cochera"],axis = 1)
testingSet.loc[:,"pileta"] = testingSet[testingSet["servicios"].notnull()].apply(lambda x: x.servicios["pileta"],axis=1)
testingSet.loc[:,"vigilancia"] = testingSet[testingSet["servicios"].notnull()].apply(lambda x: x.servicios["vigilancia"],axis=1)
testingSet.loc[:,"gimnasio"] = testingSet[testingSet["servicios"].notnull()].apply(lambda x: x.servicios["gimnasio"],axis=1)
testingSet.loc[:,"patio"] = testingSet[testingSet["servicios"].notnull()].apply(lambda x: x.servicios["patio"],axis = 1)
testingSet.loc[:,"terraza"] = testingSet[testingSet["servicios"].notnull()].apply(lambda x: x.servicios["terraza"],axis=1)
testingSet.loc[:,"parrilla"] = testingSet[testingSet["servicios"].notnull()].apply(lambda x: x.servicios["parrilla"],axis=1)
testingSet.loc[:,"lavadero"] = testingSet[testingSet["servicios"].notnull()].apply(lambda x: x.servicios["lavadero"],axis=1)

In [90]:
#Elimino la columna servicios que ya no me sirve
testingSet.drop("servicios",1,inplace=True)
testingSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 25 columns):
id                         14166 non-null int64
created_on                 14166 non-null object
property_type              14166 non-null object
operation                  14166 non-null object
place_name                 14166 non-null object
place_with_parent_names    14166 non-null object
country_name               14166 non-null object
state_name                 14166 non-null object
lat-lon                    10487 non-null object
lat                        10487 non-null float64
lon                        10487 non-null float64
surface_total_in_m2        11853 non-null float64
surface_covered_in_m2      13005 non-null float64
floor                      1368 non-null float64
rooms                      7500 non-null float64
expenses                   2543 non-null object
description                14166 non-null object
cochera                    14166 non-null int64
pileta  

In [91]:
testingSet = testingSet[testingSet['state_name'].str.contains('G.B.A') | testingSet['state_name'].str.contains('Capital Federal')]
##El tp dice solo gba y caba

#Recuperamos el dato rooms en la descripcion
testingSet.loc[testingSet.rooms.isnull(),'rooms'] = testingSet.loc[testingSet.rooms.isnull()].description.apply(recuperar_cant_amb)

#Recuperamos el dato rooms en la descripcion contando ambientes (living y dormitorios)
testingSet.loc[testingSet.rooms.isnull(),'rooms'] = testingSet.loc[testingSet.rooms.isnull()].description.apply(contar_ambientes)
testingSet.loc[testingSet.rooms == 0,'rooms'] = testingSet.loc[testingSet.rooms == 0].description.apply(contar_ambientes)


testingSet.loc[:,'place_name'] = testingSet.place_with_parent_names.apply(place_name_from_parent)
## esto quita muchos problemas de place name como nombres de country o palermo soho

In [92]:
testingSet['year'] = 0
testingSet['month'] = 0
testingSet.loc[:,'created_on'] = testingSet.created_on.apply(pd.to_datetime)
testingSet.loc[:,'year'] = testingSet.created_on.apply(lambda x: x.year)
testingSet.loc[:,'month'] = testingSet.created_on.apply(lambda x: x.month)
testingSet.drop('created_on',1,inplace=True)

In [93]:
testingSet.loc[testingSet.property_type == 'casa','property_type'] = 'house'
testingSet.loc[testingSet.property_type == 'departamento','property_type'] = 'apartment'
testingSet.loc[testingSet.property_type == 'ph','property_type'] = 'PH'

In [94]:
#Vamos a codificar cada columna que sea de strings
testingSet["property_type_code"] = lb_make1.transform(testingSet["property_type"])

testingSet["place_name_code"] = lb_make2.transform(testingSet["place_name"])

testingSet["state_name_code"] = lb_make4.transform(testingSet["state_name"])

In [95]:
trainingSet.corr()['price_aprox_usd']

price_aprox_usd                 1.000000
lat                             0.020304
lon                             0.012883
surface_total_in_m2             0.278895
rooms                           0.134659
year                            0.039029
month                           0.011079
property_type_code              0.194784
place_name_code                 0.051556
place_with_parent_names_code   -0.028929
state_name_code                -0.034208
cochera                         0.105754
pileta                          0.072432
vigilancia                      0.002261
gimnasio                       -0.001443
patio                          -0.063204
terraza                        -0.033716
parrilla                       -0.019674
lavadero                       -0.056742
Name: price_aprox_usd, dtype: float64

# Seleccion de atributos

In [97]:
data_training3 = data_training[['price_aprox_usd','price_usd_per_m2','lat','lon','surface_total_in_m2',
       'rooms', 'year', 'month', 'property_type_code','place_name_code', 
        'place_with_parent_names_code', 'state_name_code','cochera', 'pileta', 'vigilancia',
        'gimnasio', 'patio', 'terraza','parrilla', 'lavadero']]
data_training3 = data_training3.dropna(axis = 0, subset = ['lat','lon'],how ='any')

In [98]:
# Separo la columna objetivo
price_aprox_usd = data_training3['price_aprox_usd']
price_usd_per = data_training3['price_usd_per_m2']

In [99]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# Aplicando el algoritmo univariante de prueba F.
k = 12  # número de atributos a seleccionar
entrenar = data_training3.drop(['price_aprox_usd', 'price_usd_per_m2'], axis=1)
columnas = list(entrenar.columns.values)
seleccionadas = SelectKBest(f_classif, k=k).fit(entrenar, price_aprox_usd)
atrib = seleccionadas.get_support()
atributos = [columnas[i] for i in list(atrib.nonzero()[0])]
atributos

['surface_total_in_m2',
 'rooms',
 'year',
 'month',
 'property_type_code',
 'place_name_code',
 'place_with_parent_names_code',
 'state_name_code',
 'cochera',
 'pileta',
 'gimnasio',
 'parrilla']

In [100]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesRegressor
# Algoritmo de Eliminación Recursiva de atributos con ExtraTrees
entrenar = data_training3.drop(['price_aprox_usd', 'price_usd_per_m2'], axis=1)
modelo = ExtraTreesRegressor()
era = RFE(modelo, 12)  # número de atributos a seleccionar
era = era.fit(entrenar, price_aprox_usd)

In [101]:
# imprimir resultados
atrib = era.support_
atributos = [columnas[i] for i in list(atrib.nonzero()[0])]
atributos

['lat',
 'lon',
 'surface_total_in_m2',
 'rooms',
 'year',
 'month',
 'property_type_code',
 'place_name_code',
 'place_with_parent_names_code',
 'state_name_code',
 'cochera',
 'pileta']

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121146 entries, 0 to 264017
Data columns (total 19 columns):
price_usd_per_m2                121146 non-null float64
lat                             121146 non-null float64
lon                             121146 non-null float64
surface_total_in_m2             121146 non-null float64
rooms                           121146 non-null float64
year                            121146 non-null int64
month                           121146 non-null int64
property_type_code              121146 non-null int64
place_name_code                 121146 non-null int64
place_with_parent_names_code    121146 non-null int64
state_name_code                 121146 non-null int64
cochera                         121146 non-null float64
pileta                          121146 non-null float64
vigilancia                      121146 non-null float64
gimnasio                        121146 non-null float64
patio                           121146 non-null float64
terra

# Decision trees

In [106]:
######### Cambia de valor la precision cada vez que corre no tengo ni put idea por que


from sklearn import tree
dataframe = data_training3.drop(['price_aprox_usd','price_usd_per_m2'],axis=1)

columnas = list(dataframe.columns)
columnas_precio = columnas + ['price_aprox_usd']

set_entrenamiento = data_training3.loc[(data_training3.year >= 2014) &((data_training3.year < 2017) | (data_training3.month < 6)),columnas_precio]
set_pruebas = data_training3.loc[(data_training3.year == 2017) & (data_training3.month == 6),columnas_precio]

array = set_entrenamiento.values
X = array[:,0:18]
Y = array[:,18]
clf = tree.DecisionTreeRegressor()
clf = clf.fit(X, Y)
score = clf.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) *100
print("Precision = {:.2f} % , name = {}".format(score,"DecisionTreeRegressor"))


Precision = 31.12 % , name = DecisionTreeRegressor


In [103]:
from sklearn.model_selection import cross_val_score
X, y = data_training3, data_training3.price_aprox_usd
features = columnas
from sklearn.tree import DecisionTreeRegressor
regression_tree = tree.DecisionTreeRegressor(min_samples_split=30, min_samples_leaf=10,random_state=0)
regression_tree.fit(X,y)
precission = regression_tree.score(X,y) * 100
mse = np.mean(cross_val_score(regression_tree, X, y,scoring='mean_squared_error', cv=2,  n_jobs=1))
print ('Mean squared error: %.3f' % abs(mse))
print ('Precission: %.3f' % abs(precission))

/home/lucio/Documentos/OrgaDeDatos/ENV2/local/lib/python2.7/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


Mean squared error: 15572879212.045
Precission: 92.321


/home/lucio/Documentos/OrgaDeDatos/ENV2/local/lib/python2.7/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
